In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime

#need these libraries to extract P/E data from table in MarketWatch
import requests
from bs4 import BeautifulSoup
import re

#need plotly to run plots in dash
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

In [2]:
startdt=datetime.datetime(1997,1,1)
enddt=datetime.date.today()

In [3]:
ticker=['URTH','SPY','QQQ','VGK','AAXJ','EPHE']
name=['iShares MSCI World ETF',
      'SPDR S&P 500 ETF Trust',
      'Invesco QQQ Trust ',
      'Vanguard FTSE Europe Index Fund ETF Shares',
      'iShares MSCI All Country Asia ex Japan ETF',
      'iShares MSCI Philippines ETF']

In [4]:
df=pd.DataFrame([ticker,name]).transpose()
df.columns=['ticker','name']
df

,ticker,name
0,URTH,iShares MSCI World ETF
1,SPY,SPDR S&P 500 ETF Trust
2,QQQ,Invesco QQQ Trust
3,VGK,Vanguard FTSE Europe Index Fund ETF Shares
4,AAXJ,iShares MSCI All Country Asia ex Japan ETF
5,EPHE,iShares MSCI Philippines ETF


In [5]:
def extract_close_price(ticker,startdt,enddt):
    price=web.DataReader(ticker, data_source='yahoo',start=startdt, end=enddt)[['Close']]
    price.columns=[ticker]
    return price.iloc[:,0]

In [38]:
stockprice=[extract_close_price(x,startdt,enddt) for x in df.ticker]

#From List type to DataFrame
#URTH inception date 2012 hence cut in data
stockprice_df=pd.DataFrame(stockprice).transpose()
stockprice_df.dropna(inplace=True)
stockprice_df.reset_index(inplace=True)
stockprice_df.head()

,Date,URTH,SPY,QQQ,VGK,AAXJ,EPHE
0,2012-01-12,50.299999,129.509995,58.389999,41.959999,52.000000,25.370001
1,2012-01-13,50.299999,128.839996,58.180000,41.130001,51.770000,25.049999
2,2012-01-17,50.299999,129.339996,58.709999,41.900002,52.619999,25.309999
3,2012-01-18,50.299999,130.770004,59.490002,42.799999,53.580002,25.950001
4,2012-01-19,51.779999,131.460007,59.860001,43.419998,54.279999,25.950001


In [7]:
last_price=pd.DataFrame(stockprice_df.iloc[-1][1:])
last_price.reset_index(inplace=True)
last_price.columns=['ticker','last price']
last_price

,ticker,last price
0,URTH,105.04
1,SPY,357.7
2,QQQ,302.76
3,VGK,55.18
4,AAXJ,79.07
5,EPHE,25.81


In [101]:
dashboard=pd.merge(df,last_price,on='ticker')
dashboard

,ticker,name,last price
0,URTH,iShares MSCI World ETF,105.04
1,SPY,SPDR S&P 500 ETF Trust,357.7
2,QQQ,Invesco QQQ Trust,302.76
3,VGK,Vanguard FTSE Europe Index Fund ETF Shares,55.18
4,AAXJ,iShares MSCI All Country Asia ex Japan ETF,79.07
5,EPHE,iShares MSCI Philippines ETF,25.81


### Extract the P/E

Limited usefulness since I can't find a source that allows extract of historical P/E, from which I would've been able to create a Z-score or a freq. distribution of P/Es.  
It can however still tell us which markets are more expensive.

In [9]:
url_start='https://money.cnn.com/quote/etf/etf.html?symb='
url_list=[url_start+i for i in ticker]
url_list

['https://money.cnn.com/quote/etf/etf.html?symb=URTH',
 'https://money.cnn.com/quote/etf/etf.html?symb=SPY',
 'https://money.cnn.com/quote/etf/etf.html?symb=QQQ',
 'https://money.cnn.com/quote/etf/etf.html?symb=VGK',
 'https://money.cnn.com/quote/etf/etf.html?symb=AAXJ',
 'https://money.cnn.com/quote/etf/etf.html?symb=EPHE']

In [10]:
def extract_pe_ratio(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    tables=soup.find('table')
    pattern=re.compile("wsod_quoteDataPoint")
    table_data=[]
    for item in soup.find_all("td",pattern):
        table_data.append(item)
    pe_ratio=str(table_data[21])
    pattern_pe=re.compile(">(.*)</td>")
    pe=pattern_pe.search(pe_ratio).group(1)
    return pe

In [11]:
pe_ratios=[extract_pe_ratio(url) for url in url_list]
pe_ratios=pd.DataFrame(pe_ratios)
pe_ratios.columns=['PE Ratio']

In [102]:
dashboard=pd.concat([dashboard,pe_ratios],axis=1)
dashboard

,ticker,name,last price,PE Ratio
0,URTH,iShares MSCI World ETF,105.04,21.9
1,SPY,SPDR S&P 500 ETF Trust,357.7,25.0
2,QQQ,Invesco QQQ Trust,302.76,33.0
3,VGK,Vanguard FTSE Europe Index Fund ETF Shares,55.18,18.0
4,AAXJ,iShares MSCI All Country Asia ex Japan ETF,79.07,16.4
5,EPHE,iShares MSCI Philippines ETF,25.81,12.6


### Add Moving Average as Technical Trend Indicators

In [39]:
stockprice=stockprice_df.set_index('Date')
stockprice.head()

,URTH,SPY,QQQ,VGK,AAXJ,EPHE
Date,,,,,,
2012-01-12,50.299999,129.509995,58.389999,41.959999,52.000000,25.370001
2012-01-13,50.299999,128.839996,58.180000,41.130001,51.770000,25.049999
2012-01-17,50.299999,129.339996,58.709999,41.900002,52.619999,25.309999
2012-01-18,50.299999,130.770004,59.490002,42.799999,53.580002,25.950001
2012-01-19,51.779999,131.460007,59.860001,43.419998,54.279999,25.950001


In [98]:
# Get 100- and 200-day Moving Averages
MA_100=stockprice.rolling(100).mean()[99:]
MA_100.columns=["100-day MA "+ ticker for ticker in stockprice.columns]
MA_200=stockprice.rolling(200).mean()[199:]
MA_200.columns=["200-day MA "+ ticker for ticker in stockprice.columns]
MA_200.head(2)

,200-day MA URTH,200-day MA SPY,200-day MA QQQ,200-day MA VGK,200-day MA AAXJ,200-day MA EPHE
Date,,,,,,
2012-10-25,53.3947,137.8414,65.10725,44.18405,54.59770,28.79735
2012-10-26,53.4172,137.9006,65.14205,44.20425,54.61965,28.82755


In [127]:
# Get latest MA
latest_MA100=MA_100.iloc[-1].reset_index().iloc[:,1]
latest_MA200=MA_200.iloc[-1].reset_index().iloc[:,1]

#Combine
MA=pd.concat([latest_MA100,latest_MA200],axis=1)
MA.columns=['100-day MA','200-day MA']
dashboard1=pd.concat([dashboard,MA],axis=1)
dashboard1=dashboard1.round(decimals=2)

#Add Trend Indicators
trend_MA100=['UP' if i>j else 'DOWN' for i,j in zip(dashboard1['last price'],dashboard1['100-day MA'])]
trend_MA200=['UP' if i>j else 'DOWN' for i,j in zip(dashboard1['last price'],dashboard1['200-day MA'])]

#Combine in Dashboard
Trend=pd.concat([latest_MA100,pd.DataFrame(trend_MA100),latest_MA200,pd.DataFrame(trend_MA200)],axis=1)
Trend.columns=['100-day MA','Trend_100','200-day MA','Trend_200']
dashboard2=pd.concat([dashboard,Trend],axis=1)
dashboard2=dashboard2.round(decimals=2)
dashboard2

,ticker,name,last price,PE Ratio,100-day MA,Trend_100,200-day MA,Trend_200
0,URTH,iShares MSCI World ETF,105.04,21.9,92.92,UP,92.84,UP
1,SPY,SPDR S&P 500 ETF Trust,357.7,25.0,312.00,UP,308.48,UP
2,QQQ,Invesco QQQ Trust,302.76,33.0,247.29,UP,228.01,UP
3,VGK,Vanguard FTSE Europe Index Fund ETF Shares,55.18,18.0,50.31,UP,51.88,UP
4,AAXJ,iShares MSCI All Country Asia ex Japan ETF,79.07,16.4,70.30,UP,69.52,UP
5,EPHE,iShares MSCI Philippines ETF,25.81,12.6,26.17,DOWN,28.33,DOWN


In [99]:
# For charting later on, resize all and combine
stockprice_withMA=pd.concat([stockprice[199:],MA_100[100:],MA_200],axis=1)
stockprice_withMA=stockprice_withMA.round(decimals=2)
stockprice_withMA.head(2)

,URTH,SPY,QQQ,VGK,AAXJ,EPHE,100-day MA URTH,100-day MA SPY,100-day MA QQQ,100-day MA VGK,100-day MA AAXJ,100-day MA EPHE,200-day MA URTH,200-day MA SPY,200-day MA QQQ,200-day MA VGK,200-day MA AAXJ,200-day MA EPHE
Date,,,,,,,,,,,,,,,,,,
2012-10-25,54.8,141.43,65.16,45.97,56.90,31.49,53.45,139.79,66.16,44.24,53.87,29.83,53.39,137.84,65.11,44.18,54.60,28.80
2012-10-26,54.8,141.35,65.35,46.00,56.39,31.41,53.50,139.88,66.18,44.29,53.92,29.87,53.42,137.90,65.14,44.20,54.62,28.83


### Create the dashboard

In [130]:
app = JupyterDash(__name__)
#ticker and name were defined above
price=[round(num,2) for num in list(dashboard2['last price'])]
pe_ratio=list(dashboard2['PE Ratio'])
MA_100_=list(dashboard2['100-day MA'])
Trend_100_=list(dashboard2['Trend_100'])
MA_200_=list(dashboard2['200-day MA'])
Trend_200_=list(dashboard2['Trend_200'])

# Create EQ dashboard table
fig0=go.Figure(data=[go.Table(
    columnwidth=[8,30,10,10,10,10,10,10],
    header=dict(values=dashboard2.columns,fill_color='#2384AF',align='center',
                font=dict(color='white',size=12)),
                         cells=dict(values=[ticker,name,price,pe_ratio,MA_100_,Trend_100_,MA_200_,Trend_200_],
                                    fill_color='#F2F2F2',
                                    align=['center','left','center','center','center','center','center','center'],
                                   height=25))])

app.layout = html.Div([
    
    html.Div([
        dcc.Graph(
        id='equity_dashboard',
        style={'height':375},
        figure=fig0)],
    ),
    
    
    html.Div([
        dcc.Dropdown(
            id='price_graph',
            options=[{'label': i, 'value': j} for i,j in zip(name,ticker)],
            value='SPY',
            multi=True),
        dcc.Graph(id='line_graph',style={"width":"50%",'display': 'inline-block'}),
        dcc.Graph(id='histogram',style={"width":"50%",'display': 'inline-block'})
    ])
])

@app.callback(
    [Output('line_graph', 'figure'),
    Output('histogram','figure')],
    [Input('price_graph', 'value')])

def update_graph(ticker_name):
    fig1 = px.line(stockprice_df, x='Date',
                     y=ticker_name, title="Stock Price Level")
    fig2= px.histogram(stockprice_df,x=ticker_name,title="Price Frequency Distribution (2012 onwards)")
    
    return fig1, fig2

app.run_server(mode='external',port=8090)

Dash app running on http://127.0.0.1:8090/
